<a href="https://colab.research.google.com/github/JJJuniorDev/ML-colab/blob/main/batch_gd_on_iris.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sklearn.datasets import load_iris
iris= load_iris(as_frame=True)

In [6]:
iris.target.head(5)

,target
0,0
1,0
2,0
3,0
4,0


In [8]:
from numpy import int64
iris.target=iris.target.astype(int64)
iris.target_names


array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

In [40]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

# -----------------------
# 1. DATI
# -----------------------
iris = load_iris()
X = iris.data            # (150, 4)
y = iris.target          # (150,)

# train / val / test
#70% train e 30% test da parte
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.4, random_state=42
)
#30% del test diviso in 15% val e 15% test
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42
)

# -----------------------
# 2. DIMENSIONI
# -----------------------
n_features = X_train.shape[1]
n_classes = len(np.unique(y_train))

# -----------------------
# 3. MODELLO (PARAMETRI)
# -----------------------
W = np.random.randn(n_features, n_classes)
b = np.zeros(n_classes)

# -----------------------
# 4. FUNZIONI
# -----------------------
def softmax(z):
    exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))
    return exp_z / np.sum(exp_z, axis=1, keepdims=True)

def forward(X, W, b):
    scores = X @ W + b
    return softmax(scores)

def cross_entropy(y_true, y_proba):
    m = y_true.shape[0]
    return -np.mean(np.log(y_proba[np.arange(m), y_true]))

def compute_gradients(X, y, y_proba):
    m = X.shape[0]
    Y_one_hot = np.eye(n_classes)[y]

    grad_scores = (y_proba - Y_one_hot) / m
    grad_W = X.T @ grad_scores
    grad_b = np.sum(grad_scores, axis=0)

    return grad_W, grad_b

# -----------------------
# 5. TRAINING
# -----------------------
learning_rate = 0.1
n_epochs = 1000
patience = 20

best_loss = np.inf
counter = 0
best_W = W.copy()
best_b = b.copy()

for epoch in range(n_epochs):

    # TRAIN
    y_proba_train = forward(X_train, W, b)
    train_loss = cross_entropy(y_train, y_proba_train)

    grad_W, grad_b = compute_gradients(X_train, y_train, y_proba_train)

    W -= learning_rate * grad_W
    b -= learning_rate * grad_b

    # VALIDATION
    y_proba_val = forward(X_val, W, b)
    val_loss = cross_entropy(y_val, y_proba_val)

    # EARLY STOPPING
    if val_loss < best_loss:
        best_loss = val_loss
        best_W = W.copy()
        best_b = b.copy()
        counter = 0
    else:
        counter += 1
        if counter == patience:
            print(f"Early stopping at epoch {epoch}")
            break

# -----------------------
# 6. TEST
# -----------------------
def accuracy(X, y, W, b):
    y_pred = np.argmax(forward(X, W, b), axis=1)
    return np.mean(y_pred == y)

print("Test accuracy:", accuracy(X_test, y_test, best_W, best_b))


Test accuracy: 0.9666666666666667
